In [12]:
import torch
from transformers import AutoTokenizer, AutoModel
import transformers
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
# model = AutoModel.from_pretrained("bigscience/bloom-560m")

model = BloomForCausalLM.from_pretrained("bigscience/bloom-1b7")
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-1b7")

In [23]:
# prompt = "The tree has yellow flowers and green leaves."
prompt = "It was a dark and stormy night"
# result_length calibrates the size of the response (in tokens) we get for the prompt from the model
result_length = 50
inputs = tokenizer(prompt, return_tensors="pt")

In [24]:
# Greedy Search
print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_length=result_length
                      )[0]))

It was a dark and stormy night, and the wind was blowing hard. The
snow was falling fast, and the ground was covered with it. The
horses were all frozen to the ground, and the men were huddled


In [25]:
# Beam Search
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length, 
                       num_beams=2, 
                       no_repeat_ngram_size=2,
                       early_stopping=True
                      )[0]))

It was a dark and stormy night, and the wind was blowing hard. I was in the
middle of the road, when I heard a loud crash. It came from the house
at the other side of my road. A man was


In [26]:
# Sampling Top-k + Top-p
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length, 
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

It was a dark and stormy night, and the wind was blowing hard, and the
light came in only by the opening of the gate, but I could hear the
tinkling of the swords and the cries of the wild be
